In [8]:
from numpy import dot
from numpy.linalg import norm
import numpy as np
import pandas as pd 
from pandas import DataFrame
import json 
import cx_Oracle
import os
# with open('레시피 기본 정보.json','r',encoding='utf-8') as f:
#     json_data = json.load(f)
# with open('레시피 재료 정보.json','r',encoding='utf-8') as a:
#     json_data2 = json.load(a)
# with open('회원2.json','r',encoding='utf-8') as b:
#     json_data3 = json.load(b)
    
# ensure_ascii=False를 추가하니 한글로 표현됨 
# json.dumps(json_data, ensure_ascii=False,indent=4)
# json.dumps(json_data2, ensure_ascii=False,indent=4)
# json.dumps(json_data3, ensure_ascii=False, indent=4)

In [9]:
# # 리스트 출력하는 대로 출력하면 값이 잘 나타남 
# recipe = json_data["data"]
# recipe2 = json_data2["data"]
# member = json_data3["data"]

# for i in range(10):
#     print(recipe[i]['RECIPE_NM_KO'])
#     print(recipe[i]['TY_CODE'])
#     print(recipe[i]['SUMRY'])
#     print(recipe[i]['DET_URL']) # 여기에는 크롤링한 URL 추가 
#     print()
#     #spring으로 구현할 수 있을지?

In [10]:
# result_df = DataFrame(recipe,columns=['RECIPE_ID','RECIPE_NM_KO','TY_CODE','TY_NM','RATING'])
# result_df2 = DataFrame(recipe2, columns=['RECIPE_ID','IRDNT_NM'])
# result_df3 = DataFrame(member, columns=['mem_id','rating','RECIPE_ID'])

# print(result_df.head())
# print(result_df2.head())
# print(result_df3.head())

In [11]:
# data = pd.merge(result_df,result_df2, on='RECIPE_ID', how='outer')
# data.head()

# data = pd.merge(data,result_df3, on ='RECIPE_ID', how='outer')
# data.head()

# matrix=data.pivot_table(index='mem_id',columns='RECIPE_NM_KO',
#                        values='rating')
# matrix.head()

In [12]:


# doc1=np.array([0,1,1,1,0,0,0,0,0,0,0,0,0])
# doc2=np.array([1,0,1,1,1,2,5,2,0,1,2,0,0])
# doc3=np.array([2,0,2,2,4,5,1,0,4,2,1,1,2])
# doc4=np.array([0,0,0,0,0,0,0,0,0,0,0,0,0])
# doc5=np.array([0.5,0.5,0,0,5,1,2,1,0,0,2,3,0])

# print(cos_sim(doc1, doc2)) #문서1과 문서2의 코사인 유사도
# print(cos_sim(doc1, doc3)) #문서1과 문서3의 코사인 유사도
# print(cos_sim(doc2, doc3))

# print('-'*50)
# print(cos_sim(doc5, doc2))

# #if(np.isnan(cos_sim(doc4, doc5))):
#  #   print("레시피를 추천할 수 없습니다. 레시피 평가를 해주세요.")

# # 나의 레시피 점수
# #re_list = [1,2,3,0]
# #for i in range 

# # 모든 사람의 레시피 점수 (해결)
# # 리스트를 출력 
# # re_list랑 i[] 와 쭉쭉 비교 
# #list1 = [] 
# #for i in range(?):
# #    list1[i] = cos_sim(re_list, i[])
# #    list1.max
#     #여기서 max값이 뭐랑 비교했는지 어떻게 알지 
#     # cnt 값을 추출해야지 
    
#     # cnt값이랑 re_list의 빈값 비교후 if문 
#     # 리스트에 없는 값 추출 후 
#     # 내림차순으로 추천 
#     # ok


In [13]:
os.putenv('NLS_LANG', '.UTF8')
#연결에 필요한 기본 정보(유저, 비밀번호, 데이터베이스 서버 주소)
#connection = cx_Oracle.connect('admin','operwhe10','database-2.c2xjfsjmjzmt.ap-northeast-2.rds.amazonaws.com:1521/ORCL')
conn = cx_Oracle.connect("admin/operwhe10@database-2.c2xjfsjmjzmt.ap-northeast-2.rds.amazonaws.com:1521/ORCL")
cursor = conn.cursor()

# 코싸인 유사도 함수 생성
def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))

# 함수 생성 (튜플 -> 딕셔너리 )
def makeDictFactory(cursor):
   columnNames = [d[0] for d in cursor.description]
 
 
   def createRow(*args):
      return dict(zip(columnNames, args))
 
 
   return createRow

# 로그인한 사람의 평가를 제거하는 함수 
# 만약 평가가 제거되지 않으면 비교대상인 자신과 유사도가 가장 높으므로
# 올바른 추천을 할 수 없음

def comparison(array,list):
    list1 = []
    for m in list:
        if(np.array_equal(array,m)):
            continue
        else:
            list1.append(m)
    return list1

# mem_evaluation(멤버id, url, rating, 레시피id) 테이블 불러오기
cursor.execute("select * from mem_evaluation")
# 튜플값을 dic값으로 바꿈
cursor.rowfactory = makeDictFactory(cursor)

# MEM_ID가 imgentleman01인 값을 불러오기 (이건 테스트용, 아래 코드에서 사용되지는 않음.)
# => 따라서 차후 홈페이지 구현 시 sql문은 로그인한 회원 아이디를 불러와야한다.
cursor.execute("select * from mem_evaluation where MEM_ID = 'imgentleman01'")
# 튜플 -> 딕 
cursor.rowfactory = makeDictFactory(cursor)
# 값을 rows에 저장
rows = cursor.fetchall()

# list2를 생성 
list2 = []

# for문으로 imgentleman01이 평가한 레이팅 값만 불러와 list2에 저장
for row in rows :
    list2.append(int(row['RATING']))
    
#출력
print(np.array(list2))
# 아래와 같이 db에 저장된 rating값을 array로 만들어낼 수 있다.

[3 0 4 5 0]


In [14]:
# cursor.execute("select RATING, RECIPE_ID from mem_evaluation1 where SEQ = 1 order by MEM_ID" )
# cursor.rowfactory = makeDictFactory(cursor)
# a=cursor.fetchall()
# mylist=[]

# 여기서부터 구현 프로그램 
# 목표 :: 무한루프를 돌리되, 한 아이디의 레이팅 값이 끝나면 
#        다음 아이디로 넘어가서 반복한다. 이후 더이상 값이 없으면 무한루프 탈출.

# 아이디값을 저장한 후, 
cursor.execute("select MEM_ID from mem_evaluation order by MEM_ID")
i = cursor.fetchall()

# 튜플을 리스트로 만든 후, 중복값을 제거한다.
# => 즉, 아이디가 중복되지 않게 리스트에 저장한다.
i = list(i)
i= list(set(i))
# 정렬 
i.sort()


#ID와 RATING값을 저장한다. (ID값 기준으로 )
cursor.execute("select MEM_ID, RATING from mem_evaluation order by MEM_ID")
cursor.rowfactory = makeDictFactory(cursor)
j = cursor.fetchall()

# 레시피 테이블을 불러와 recipe에 저장한다.
cursor.execute("select RECIPE_ID, RECIPE_NM_KO from recipe_info")
recipe = cursor.fetchall()
# 값이 정렬이 안 되어 있으므로, 오름차순으로 정렬 
recipe.sort()

# i와 j의 값이 같은지 확인한다. (str로 변환 필요)
if(str(i[0][0])==str(j[0]['MEM_ID'])):
     print("test")

# 리스트 생성 후, 레이팅 값을 각각 담는다
test_list = []
cursor.execute("select count(RATING) FROM mem_evaluation")
ok = cursor.fetchall()

# 튜플값으로 저장되므로, 값만 꺼내오기위해 [0][0]
ok= int(ok[0][0])

# 카운트 값을 기준으로 for문을 돌려 list에 담는다.
for m in range(ok):
     test_list.append(j[m]['RATING'])
print(np.array(test_list))


test
[4 0 3 0 5 5 2 3 3 3 1 2 5 3 1 2 4 5 1 3 5 2 3 2 2]


In [15]:
fi_list=[]
for m in i:
    fi_list.append(m[0])
print(fi_list)

# 변수 q=0 
q=0
final_list=[]
# for문으로 fi_list만큼 돌린다.
for m in range(len(fi_list)):
    list1=[]
    while(q<len(j)):
        # fi_list의 값과 j의 'MEM_ID'가 같다면
        if(str(fi_list[m])==str(j[q]['MEM_ID'])):
            
            #해당 아이디가 평가한 RATING값을 list1에 저장한다.
            list1.append(j[q]['RATING'])
            # 순환을 위해 q값을 추가
            q=q+1
            
            # q값이 전체 j의 숫자와 같다면
            if(q==len(j)):
                # final_list list1의 값을 담는다.
                # 이후는 for문의 조건식에 맞지 않으므로 빠져나감.
                final_list.append(list1)
        
        # 다르다면 final_list에 담고 빠져나간다.
        # id값이 처음부터 다르다면 값이 없으므로, 담는 값이 없는 것.
        else:
            final_list.append(list1)
            break
            
# final_list 확인            
print(final_list)
ffinal_list=[]

# 담긴 final_list를 numpy의 rray 리스트로 만들어 ffinal_list에 저장한다.
 
for m in range(len(final_list)):
    ffinal_list.append(np.array(final_list[m]))
print('-'*50)
# ffinal_list 확인 ( Array로 된 것을 확인할 수 있다.)
print(ffinal_list)

['imgentleman01', 'imgentleman02', 'imgentleman03', 'imgentleman04', 'imgentleman05']
[[4, 0, 3, 0, 5], [5, 2, 3, 3, 3], [1, 2, 5, 3, 1], [2, 4, 5, 1, 3], [5, 2, 3, 2, 2]]
--------------------------------------------------
[array([4, 0, 3, 0, 5]), array([5, 2, 3, 3, 3]), array([1, 2, 5, 3, 1]), array([2, 4, 5, 1, 3]), array([5, 2, 3, 2, 2])]


In [16]:
doc_list=[]
for n in ffinal_list:
    doc_list.append(np.array(n))

# 임의의 array 리스트를 하나 만든 것
# 차후에는 여기에 로그인한 회원의 Rating이 들어갈 것
doc1 = np.array([1,0,0,5,3])

# 중복 아이디를 제거한다.
real_final_list = comparison(doc1,doc_list)
print(real_final_list)

# 임의로 변수에 담아본 것
A=doc1

# 회원별 코싸인 유사도를 담을 리스트
com_list=[]

# for문을 이용하여 A(로그인한 회원)와 전체 회원의 코싸인 유사도를 구한다.
for m in real_final_list:
    com_list.append(cos_sim(A,m))
print(com_list)
# ok 하나씩 비교 완료 

# 가장 유사도가 높은 회원의 인덱스를 저장한다.
max_index = com_list.index(max(com_list))
# ok 아이디 찾기 완료

# 가장 높은 회원의 레이팅 점수 리스트를 가져온다.
max_rating = real_final_list[max_index]
print(max_rating)
# ok 아이디 레이팅 찾기 완료 

print('-'*50)

#인덱스와 레이팅 점수를 담을 리스트를 각각 생성 
ok_list=[]
ok_list_index=[]

# for문을 이용하여, len(max_rating)만큼 반복한다.
# 여기서 len(max_rating) 값은 모든 회원이 같아야 함 
# => 즉 회원가입 때 가입한 회원이 기본 레이팅 값을 가져야 하는 것.
# db에서 디폴트 값으로 구현 가능할 듯?
# 추가적으로 레시피 추가 시, 모든 회원이 추가된 레시피에 기본 레이팅 값을 가져야 함
# 위 두 사항은... 무조건임 

for ok in range(len(max_rating)):
    # 유사도가 가장 높은 회원이 평가한 레시피 중 
    # 로그인한 회원이 평가하지 않은 값을 찾는다.
    if(max_rating[ok] != 0 and doc1[ok]==0):
        
        # 찾아낸 점수를 list에 저장하고 
        ok_list.append(max_rating[ok])
        # 찾아낼 당시의 ok값을 index로 활용한다.
        ok_list_index.append(ok)

print(ok_list)
print(ok_list_index)

# 최종 리스트 생성 
final_ok_list = []

print('-'* 50)
# ok_list 숫자를 기준으로 반복한다.
# 리스트 하나당 [레이팅, 인덱스]로 만들어 새로운 최종 리스트에 담는다.
for okok in range(len(ok_list)):
    list1=[] 
    list1.append(ok_list[okok])
    list1.append(ok_list_index[okok])
    final_ok_list.append(list1)
    

# 레시피 점수를 기준으로 내림차순 정렬을 한다. 
final_ok_list = sorted(final_ok_list, key = lambda x : (-x[0], x[1]))
print(final_ok_list)
print('-'* 50)
# 레시피 이름을 출력하기 위한 for문
# 추천 레시피 리스트인 final_ok_list의 length값만큼 for문을 돌린다.
cnt=1

for m in range(len(final_ok_list)):
    # 레시피 전체 조회를 위한 for문 
    for n in range(len(recipe)):
        # 만약 final_ok_list의 인덱스와 레시피의 인덱스가 같다면
        if(final_ok_list[m][1] == recipe[n][0]):
            # 출력 
            print("회원님께 드리는 %d번째 추천 레시피 : " %cnt,recipe[n][1])
            cnt = cnt+1
# 여기까지! 



[array([4, 0, 3, 0, 5]), array([5, 2, 3, 3, 3]), array([1, 2, 5, 3, 1]), array([2, 4, 5, 1, 3]), array([5, 2, 3, 2, 2])]
[0.45418687154706955, 0.6550432296063071, 0.5077963596336063, 0.36467384467084146, 0.5233670255931882]
[5 2 3 3 3]
--------------------------------------------------
[2, 3]
[1, 2]
--------------------------------------------------
[[3, 2], [2, 1]]
--------------------------------------------------
회원님께 드리는 1번째 추천 레시피 :  오곡밥
회원님께 드리는 2번째 추천 레시피 :  나물비빔밥
